## Dependencies

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import os                       # for working with files
import os
from PIL import Image
import numpy as np

In [3]:
# Verify if torch is installed
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.1+cpu
False


#  Explore the dataset

###  Load the dataset

In [12]:
data_dir = './Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented'
data = os.listdir(data_dir)
diseases = []
for crop in data:
    if os.path.isdir(data_dir+'/'+crop):
        diseases.append(os.listdir(data_dir+'/'+crop+'/test_set'))
diseases

[['anthracnose', 'gumosis', 'healthy', 'leaf miner', 'red rust'],
 ['bacterial blight', 'brown spot', 'green mite', 'healthy', 'mosaic'],
 ['fall armyworm',
  'grasshoper',
  'healthy',
  'leaf beetle',
  'leaf blight',
  'leaf spot',
  'streak virus'],
 ['healthy',
  'leaf blight',
  'leaf curl',
  'septoria leaf spot',
  'verticulium wilt']]

In [13]:
print("Total disease classes are: {}".format(len(diseases)))

Total disease classes are: 4
